In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unsw-nb15/UNSW_NB15_testing-set.csv
/kaggle/input/unsw-nb15/UNSW-NB15_1.csv
/kaggle/input/unsw-nb15/UNSW_NB15_training-set.csv
/kaggle/input/unsw-nb15/UNSW-NB15_LIST_EVENTS.csv
/kaggle/input/unsw-nb15/UNSW-NB15_4.csv
/kaggle/input/unsw-nb15/UNSW-NB15_3.csv
/kaggle/input/unsw-nb15/UNSW-NB15_2.csv
/kaggle/input/unsw-nb15/NUSW-NB15_features.csv


In [2]:
!pip install -q pandas numpy scikit-learn matplotlib seaborn plotly yfinance tqdm
!pip install -q torch torchvision
!pip install -q opendatasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have 

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances
from tqdm import tqdm
import plotly.express as px

In [4]:
dataset_path = '/kaggle/input/unsw-nb15/UNSW_NB15_testing-set.csv'  # Change if needed

# Load dataset
df = pd.read_csv(dataset_path)
print("Original shape:", df.shape)

# Identify categorical and numeric columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns
num_cols = df.select_dtypes(include=[np.number]).columns

# Encode categorical features
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Normalize the entire dataset
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled = df_scaled.clip(0, 1)

print("Preprocessed shape:", df_scaled.shape)
df_scaled.head()


Original shape: (175341, 45)
Preprocessed shape: (175341, 45)


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000000,0.002025,0.856061,0.00,0.25,0.000520,0.000364,0.000018,0.000012,0.000074,...,0.0,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.666667,0.0
1,0.000006,0.010832,0.856061,0.00,0.25,0.001352,0.003463,0.000054,0.002867,0.000078,...,0.0,0.015625,0.00,0.00,0.0,0.000000,0.081967,0.0,0.666667,0.0
2,0.000011,0.027052,0.856061,0.00,0.25,0.000728,0.001458,0.000026,0.000900,0.000014,...,0.0,0.031250,0.00,0.00,0.0,0.016949,0.081967,0.0,0.666667,0.0
3,0.000017,0.028027,0.856061,0.25,0.25,0.001144,0.001093,0.000046,0.000053,0.000014,...,0.0,0.031250,0.25,0.25,0.0,0.016949,0.000000,0.0,0.666667,0.0
4,0.000023,0.007491,0.856061,0.00,0.25,0.000936,0.000547,0.000039,0.000018,0.000033,...,0.0,0.609375,0.00,0.00,0.0,0.016949,0.622951,0.0,0.666667,0.0


In [5]:
class TabularDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [6]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
        nn.Linear(input_dim, 128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(256, output_dim),
        nn.Sigmoid()
        )


    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3),
        nn.Linear(256, 128),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.3),
        nn.Linear(128, 1),
        nn.Sigmoid()
    )

    def forward(self, x):
        return self.model(x)


In [7]:
batch_size = 512
epochs = 300
noise_dim = 100
generated_sample_count = 10000  # Customize this

input_dim = df_scaled.shape[1]
dataset = TabularDataset(df_scaled)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

G = Generator(noise_dim, input_dim)
D = Discriminator(input_dim)

criterion = nn.BCELoss()
# G_optimizer = optim.Adam(G.parameters(), lr=0.0003)
# D_optimizer = optim.Adam(D.parameters(), lr=0.0003)
G_optimizer = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))


G.train()
D.train()

for epoch in range(epochs):
    for real_data in dataloader:
        batch_size = real_data.size(0)

        # Train Discriminator
        # real_labels = torch.ones(batch_size, 1)
        # fake_labels = torch.zeros(batch_size, 1)
        real_labels = torch.ones(batch_size, 1) * 0.9  # Label smoothing
        fake_labels = torch.zeros(batch_size, 1)


        outputs = D(real_data)
        d_loss_real = criterion(outputs, real_labels)

        z = torch.randn(batch_size, noise_dim)
        fake_data = G(z)
        outputs = D(fake_data.detach())
        d_loss_fake = criterion(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake

        D_optimizer.zero_grad()
        d_loss.backward()
        torch.nn.utils.clip_grad_norm_(D.parameters(), max_norm=1.0)

        D_optimizer.step()

        # Train Generator
        z = torch.randn(batch_size, noise_dim)
        fake_data = G(z)
        outputs = D(fake_data)
        g_loss = criterion(outputs, real_labels)

        G_optimizer.zero_grad()
        g_loss.backward()
        torch.nn.utils.clip_grad_norm_(G.parameters(), max_norm=1.0)

        G_optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}")


Epoch [5/300], d_loss: 0.8886, g_loss: 1.4643
Epoch [10/300], d_loss: 1.0744, g_loss: 1.2460
Epoch [15/300], d_loss: 1.0475, g_loss: 1.2467
Epoch [20/300], d_loss: 0.9725, g_loss: 1.3257
Epoch [25/300], d_loss: 0.9903, g_loss: 1.2778
Epoch [30/300], d_loss: 0.9571, g_loss: 1.3560
Epoch [35/300], d_loss: 1.0239, g_loss: 1.2448
Epoch [40/300], d_loss: 1.0034, g_loss: 1.3079
Epoch [45/300], d_loss: 1.0001, g_loss: 1.3217
Epoch [50/300], d_loss: 1.0394, g_loss: 1.2240
Epoch [55/300], d_loss: 1.0335, g_loss: 1.2789
Epoch [60/300], d_loss: 1.0985, g_loss: 1.1204
Epoch [65/300], d_loss: 1.1081, g_loss: 1.0497
Epoch [70/300], d_loss: 1.1458, g_loss: 1.0668
Epoch [75/300], d_loss: 1.0662, g_loss: 1.0823
Epoch [80/300], d_loss: 1.0835, g_loss: 1.1085
Epoch [85/300], d_loss: 1.0653, g_loss: 1.1654
Epoch [90/300], d_loss: 1.0592, g_loss: 1.1814
Epoch [95/300], d_loss: 1.1226, g_loss: 1.0938
Epoch [100/300], d_loss: 1.1377, g_loss: 1.1687
Epoch [105/300], d_loss: 1.0846, g_loss: 1.1385
Epoch [110/3

In [8]:
G.eval()
synthetic_data = []

with torch.no_grad():
    for _ in tqdm(range(generated_sample_count // batch_size)):
        z = torch.randn(batch_size, noise_dim)
        synthetic_batch = G(z).detach().numpy()
        synthetic_data.append(synthetic_batch)

synthetic_data = np.vstack(synthetic_data)[:generated_sample_count]
synthetic_df = pd.DataFrame(scaler.inverse_transform(synthetic_data), columns=df.columns)
synthetic_df = synthetic_df.round(2)
synthetic_df.head()


100%|██████████| 42/42 [00:00<00:00, 1626.39it/s]


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,81085.976562,0.00,64.820000,0.00,2.99,2.86,0.0,77.820000,2.30,164060.390625,...,2.15,6.110000,0.0,0.0,0.0,15.110000,5.810000,0.0,2.28,1.0
1,171101.953125,0.01,118.449997,1.99,2.98,1.53,0.0,483.049988,4.09,339128.625000,...,16.35,31.170000,0.0,0.0,0.0,26.879999,31.059999,0.0,4.99,1.0
2,93117.468750,0.00,120.610001,0.00,3.00,1.79,0.0,55.549999,0.14,192230.500000,...,1.63,5.210000,0.0,0.0,0.0,1.940000,5.180000,0.0,2.22,1.0
3,150223.500000,0.00,118.910004,2.00,2.99,2.39,0.0,125.010002,0.50,136598.281250,...,12.93,27.219999,0.0,0.0,0.0,24.530001,27.059999,0.0,4.99,1.0
4,129443.601562,0.00,119.930000,0.00,3.00,1.53,0.0,149.289993,0.26,158132.171875,...,3.42,4.370000,0.0,0.0,0.0,4.250000,4.620000,0.0,2.47,1.0


In [9]:
combined = np.vstack([df_scaled.sample(1000).values, synthetic_data[:1000]])
labels = ["Original"] * 1000 + ["Synthetic"] * 1000

tsne = TSNE(n_components=2, perplexity=30, n_iter=1000)
tsne_result = tsne.fit_transform(combined)
tsne_df = pd.DataFrame(tsne_result, columns=['x', 'y'])
tsne_df['Type'] = labels

fig = px.scatter(tsne_df, x='x', y='y', color='Type', title='t-SNE of Original vs Synthetic Data')
fig.show()

# Jensen-Shannon Divergence
from scipy.spatial.distance import jensenshannon
jsd = jensenshannon(df_scaled.mean(), pd.DataFrame(synthetic_data).mean())
print(f"Jensen-Shannon Divergence: {jsd:.4f}")


Jensen-Shannon Divergence: 0.1352


In [11]:
# from scipy.stats import ks_2samp

# for col in df_scaled.columns:
#     stat, p_value = ks_2samp(df_scaled[col], pd.DataFrame(synthetic_data)[col])
#     if p_value < 0.05:
#         print(f"{col}: Significant difference (p={p_value:.4f})")
#     else:
#         print(f"{col}: Similar distribution (p={p_value:.4f})")


In [12]:
dists = pairwise_distances(df_scaled.values, synthetic_data)
dcr = np.min(dists, axis=0)
print(f"Average Distance to Closest Real Record: {np.mean(dcr):.4f}")


Average Distance to Closest Real Record: 0.0773


In [13]:
synthetic_df.to_csv("synthetic_data.csv", index=False)
print("Synthetic dataset saved as synthetic_data.csv")


Synthetic dataset saved as synthetic_data.csv
